<a href="https://colab.research.google.com/github/JoesHage/ML_PROJECT/blob/main/Food_Recognition_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Food Item Recognition**

We will be using the following 3 models to test which one of them is ideal to our current scenario.
*   **SVM**
*   **Random Forest**
*   **CNN**

We need to classify images into 1 of 9 classes using these models.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os

# Get the path to the data folder
data_folder = "/content/drive/MyDrive/Food Item Recognition From Images1 (1)/data"

# Get a list of all folders in the data folder
folders = pd.Series(os.listdir(data_folder)).sort_values()

# Print the list of folders
print(folders)


2               1
4               2
1               3
5               4
7               5
6               6
8               7
9               8
0               9
3    category.txt
dtype: object


# **I. Support Vector Machines**


# **II. Random Forest**

In [ ]:
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import accuracy_score

def extract_features(image_path, image_size=(100, 100)):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Unable to load image '{image_path}'")
            return None
        image = cv2.resize(image, image_size)
        # Convert image to grayscale and flatten the array
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return gray_image.flatten()
    except Exception as e:
        print(f"Error: An exception occurred while processing image '{image_path}': {e}")
        return None

X = []
y = []

with open(os.path.join(data_folder, 'category.txt'), 'r') as category_file:
    next(category_file)  # Skip the header line
    for line in category_file:
        category_id, category_name = line.strip().split('\t')
        category_id = int(category_id)
        category_folder = os.path.join(data_folder, str(category_id))
        # Iterate over images in the category folder
        for image_file in os.listdir(category_folder):
            image_path = os.path.join(category_folder, image_file)
            # Extract features and append to X
            features = extract_features(image_path)
            X.append(features)
            # Append corresponding label to y
            y.append(category_id)

X = np.array(X)
y = np.array(y)
#Split data into training and testing data, 0.8 and 0.2 respectively.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

#Train Random forest classifier on training data
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=32)
rf_classifier.fit(X_train, y_train)

#Evaluation
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

#Using 10 fold cross validation
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=32)
cv_scores = cross_val_score(rf_classifier, X, y, cv=5)
print(f"CV Accuracy Scores: {cv_scores}")
print(f"Average CV Accuracy: {np.mean(cv_scores)}")

#Using hyperParameter tuning to find the ideal estimator.
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, 30],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_classifier = RandomForestClassifier(random_state=32)

grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validated accuracy: {:.2f}".format(grid_search.best_score_))

#Testing the best model we got from the grid search's estimator
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test set accuracy: {:.2f}".format(accuracy))

Accuracy: 0.5910418695228822
CV Accuracy Scores: [0.50827653 0.60564752 0.54527751 0.59746589 0.50487329]
Average CV Accuracy: 0.552308147844457
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters found:  {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Best cross-validated accuracy: 0.59
Test set accuracy: 0.60


Using only the testing, training data we check the accuracy of the model and it gives a moderate 0.58.
Now we move on and add 10 fold cross validation for a more accurate accuracy score: 0.54.
We now want to improve the model more by using hyperparameter tuning and getting the best parameters for a model.

# **III. Convolutional neural networks**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import random
import shutil
from PIL import Image
from sklearn.model_selection import train_test_split

# Directory containing the images
data_dir = "/content/drive/MyDrive/data"
categories = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]

# Directory for train, validation, and test sets
output_dir = "output"
train_dir = os.path.join(output_dir, "train")
val_dir = os.path.join(output_dir, "validation")
test_dir = os.path.join(output_dir, "test")

# Create output directories if they don't exist
for directory in [train_dir, val_dir, test_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Helper function to check if an image is corrupted
def is_image_valid(file_path):
    try:
        img = Image.open(file_path)
        img.verify()  # Verify that it is, in fact, an image
        return True
    except (IOError, SyntaxError) as e:
        return False

# Split data and move to appropriate folders
for category in categories:
    category_path = os.path.join(data_dir, category)
    all_files = [os.path.join(category_path, f) for f in os.listdir(category_path) if is_image_valid(os.path.join(category_path, f))]

    train_files, test_files = train_test_split(all_files, test_size=0.2, random_state=42)
    val_files, test_files = train_test_split(test_files, test_size=0.5, random_state=42)

    for file_set, dest_dir in [(train_files, train_dir), (val_files, val_dir), (test_files, test_dir)]:
        dest_category_dir = os.path.join(dest_dir, category)
        if not os.path.exists(dest_category_dir):
            os.makedirs(dest_category_dir)

        for file_path in file_set:
            shutil.copy(file_path, dest_category_dir)

print("Data successfully split into training, validation, and test sets.")


Data successfully split into training, validation, and test sets.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Directory paths
batch_size = 32
img_height = 150
img_width = 150

train_gen = ImageDataGenerator(rescale=1./255, rotation_range=30, horizontal_flip=True)
val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(train_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')
val_data = val_gen.flow_from_directory(val_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')
test_data = test_gen.flow_from_directory(test_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    BatchNormalization(),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(categories), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Found 4118 images belonging to 9 classes.
Found 514 images belonging to 9 classes.
Found 519 images belonging to 9 classes.


In [ ]:
# Callbacks for preventing overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')

# Training the model
history = model.fit(train_data, validation_data=val_data, epochs=50, callbacks=[early_stop, checkpoint])


Epoch 1/50
129/129 [==============================] - ETA: 0s - loss: 13.4678 - accuracy: 0.4140

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


129/129 [==============================] - 270s 2s/step - loss: 13.4678 - accuracy: 0.4140 - val_loss: 9.9703 - val_accuracy: 0.1634
Epoch 2/50
129/129 [==============================] - 272s 2s/step - loss: 5.9478 - accuracy: 0.5036 - val_loss: 2.6637 - val_accuracy: 0.2393
Epoch 3/50
129/129 [==============================] - 268s 2s/step - loss: 2.3305 - accuracy: 0.5481 - val_loss: 4.7167 - val_accuracy: 0.4047
Epoch 4/50
129/129 [==============================] - 271s 2s/step - loss: 1.3183 - accuracy: 0.5937 - val_loss: 1.7309 - val_accuracy: 0.5156
Epoch 5/50
129/129 [==============================] - 265s 2s/step - loss: 1.0474 - accuracy: 0.6428 - val_loss: 1.3184 - val_accuracy: 0.6089
Epoch 6/50
129/129 [==============================] - 267s 2s/step - loss: 0.9691 - accuracy: 0.6819 - val_loss: 1.2381 - val_accuracy: 0.6634
Epoch 7/50
129/129 [==============================] - 264s 2s/step - loss: 0.9437 - accuracy: 0.6938 - val_loss: 1.0548 - val_accuracy: 0.6965
Epoch 8/5